In [1]:
#%pdb

In [2]:
import math
from sortedcontainers import SortedDict
import random
import numpy 
import matplotlib.pyplot as plt

In [3]:
# Code for generating positions, distances, travel times and preferences
def generate_positions(males, x_dim, y_dim):
    Xs = numpy.random.rand(males) * x_dim
    Ys = numpy.random.rand(males) * y_dim
    return [Xs,Ys]

def compute_distances_travel_times(males, positions, bird_speed):
    male_dist = numpy.zeros((males, males))
    travel_times = numpy.zeros((males, males))
    for i in range(males):
        for j in range(i + 1, males):
            dist = math.sqrt((positions[0][j] - positions[0][i]) ** 2 + (positions[1][j] - positions[1][i]) ** 2)
            travel = dist / bird_speed
            male_dist[j][i] = dist
            male_dist[i][j] = dist
            travel_times[j][i] = travel
            travel_times[i][j] = travel
    return (male_dist, travel_times)

def compute_visit_preferences(males, distances, lambda_dist):
    # compute exponential of each coefficient
    visit_preferences = numpy.exp(-lambda_dist * distances)
    # remove the identity matrix (exp(0) = 1)
    visit_preferences = visit_preferences - numpy.eye(males)
    # make rows sum to one
    visit_preferences = (visit_preferences.transpose() / numpy.sum(visit_preferences, 1)).transpose()
    return visit_preferences

In [4]:
# functions to generate tickets and manage timeline
def generate_ticket(start_time, end_time, length_activity, owner, action, target):
    global timeline
    ticket = {"start_time": start_time,
              "end_time": end_time,
              "length_activity": length_activity,
              "owner": owner,
              "action": action,
              "target": target
             }
    # now add to timeline
    timeline[(ticket["end_time"], ticket["owner"])] = ticket


In [5]:
# ACTION FUNCTIONS
# Each action generates a ticket, and updates the state of the owner (and possibly the target)

def draw_foraging_time(start_time):
    # note -- for now take the absolute value. Can be changed
    # note: HARD CODED PARAMETERS!
    next_time = numpy.abs(numpy.random.normal(loc = 0.4, scale = 0.167))
    return next_time + start_time

def action_forage(bird_id, current_time):
    global birds
    # generate the time it takes to forage
    # note: HARD CODED PARAMETERS!
    time_spent_foraging = numpy.random.gamma(shape = 1.5, scale = 5.0) / 60.0
    time_action_ends = current_time + time_spent_foraging
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_spent_foraging,
                   owner = bird_id,
                   action = "foraging",
                   target = -1)
    # update the bird:
    birds[bird_id]["current_state"] = "foraging"
    birds[bird_id]["action_starts"] = current_time
    birds[bird_id]["action_ends"] = time_action_ends
    # update the time to next foraging: start counting when foraging ended
    birds[bird_id]["next_foraging_time"] = draw_foraging_time(time_action_ends)

def action_stay_at_bower(bird_id, current_time):
    global birds
    # generate the length of the stay
    # notes: Taking abs; HARD CODED PARAMS!
    time_spent_at_bower = numpy.abs(numpy.random.normal(loc = 0.1583, scale= 0.09755))
    time_action_ends = current_time + time_spent_at_bower
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_spent_at_bower,
                   owner = bird_id,
                   action = "staying at bower",
                   target = -1)
    # update the bird:
    birds[bird_id]["current_state"] = "staying at bower"
    birds[bird_id]["action_starts"] = current_time
    birds[bird_id]["action_ends"] = time_action_ends

def action_travel_to_maraud(bird_id, current_time):
    global birds
    # choose who to maraud
    tmp = numpy.random.rand()
    target = numpy.argwhere(birds[bird_id]["travel_preferences"] > tmp)[0][0] 
    time_to_travel = birds[bird_id]["travel_times"][target]
    time_action_ends = current_time + time_to_travel
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_to_travel,
                   owner = bird_id,
                   action = "travel to maraud",
                   target = target)
    # update the bird:
    birds[bird_id]["current_state"] = "travel to maraud"
    birds[bird_id]["action_starts"] = current_time
    birds[bird_id]["action_ends"] = time_action_ends

def action_repair_bower(bird_id, current_time):
    global birds
    # generate the length of the repair bout
    # notes: Taking abs; HARD CODED PARAMS!
    time_spent_repairing_bower = numpy.abs(numpy.random.normal(loc = 0.1583, scale = 0.09755))
    time_action_ends = current_time + time_spent_repairing_bower
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_spent_repairing_bower,
                   owner = bird_id,
                   action = "repairing bower",
                   target = -1)
    # update the bird:
    birds[bird_id]["current_state"] = "repairing bower"
    birds[bird_id]["action_starts"] = current_time
    birds[bird_id]["action_ends"] = time_action_ends
    # note: already accounts for the improvements
    birds[bird_id]["bower_state"] = birds[bird_id]["bower_state"] + time_spent_repairing_bower
    # cannot make it better than 0
    if birds[bird_id]["bower_state"] > 0.0:
        birds[bird_id]["bower_state"] = 0.0

def action_maraud(marauder_id, marauder_target, current_time):
    global birds
    # note: HARD CODED PARAMS!
    time_spent_marauding = 0.1 
    # note: HARD CODED PARAMS!
    damage_to_bower = 6.0 
    time_action_ends = current_time + time_spent_marauding
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_spent_marauding,
                   owner = marauder_id,
                   action = "marauding",
                   target = marauder_target)
    # update marauder
    birds[marauder_id]["current_state"] = "marauding"
    birds[marauder_id]["action_starts"] = current_time
    birds[marauder_id]["action_ends"] = time_action_ends
    # update target
    birds[marauder_target]["bower_state"] = birds[marauder_target]["bower_state"] - damage_to_bower
    

def action_travel_from_maraud(marauder_id, marauder_target, current_time):
    global birds
    time_from_travel = birds[marauder_id]["travel_times"][marauder_target]
    time_action_ends = current_time + time_from_travel
    # generate the ticket
    generate_ticket(start_time = current_time,
                   end_time = time_action_ends,
                   length_activity = time_from_travel,
                   owner = marauder_id,
                   action = "travel from maraud",
                   target = marauder_target)
    # update the bird:
    birds[marauder_id]["current_state"] = "travel from maraud"
    birds[marauder_id]["action_starts"] = current_time
    birds[marauder_id]["action_ends"] = time_action_ends
    
def action_mating_attempt(female_id, current_time):
    global birds
    global female_birds
    last_location=female_birds[female_id]["already_visited"][-1:][0]
    print("last_loc")
    print(last_location)
    if len(female_birds[female_id]["already_visited"]) == female_birds[female_id]["max_per_day"]:
        female_birds[female_id]["already_visited"] = []
        tmp = numpy.random.rand()
        target = numpy.argwhere(birds[last_location]["travel_preferences"] > tmp)[0][0]
        time_to_travel = birds[last_location]["travel_times"][target]
        time_action_ends = current_time + time_to_travel
        generate_ticket(start_time = time_action_ends + 12, #HARD CODE
                        end_time = time_action_ends + 12,
                        length_activity = 0,
                        owner = female_id,
                        action = "mating attempt",
                        target = target)
    else:        
        p = visit_preferences[last_location] #list of preferences by index (has probability -- not cumulative)
        #print(p)
        av=female_birds[female_id]["already_visited"]
        #print(av)
        p[av] = 0.0
        p = numpy.cumsum(p)
        print("p:")
        print(p)
        scale_rand = p[-1:]
        tmp = numpy.random.rand() * scale_rand
        #print(p)
        target = numpy.argwhere(p > tmp)[0][0]
        time_to_travel = birds[last_location]["travel_times"][target]
        time_action_ends = current_time + time_to_travel
        generate_ticket(start_time = time_action_ends, #HARD CODE
                        end_time = time_action_ends,
                        length_activity = 0,
                        owner = female_id,
                        action = "mating attempt",
                        target = target)
        

In [6]:
# this function should be called every time the bird 
# 1) is back to the bower (from foraging, marauding), 
# 2) has finished repairing the bower, 
# 3) or has finished a stint at staying at bower
def choose_action(bird, current_time):
    global t_max
    # stop generating actions at t_max
    if current_time < t_max:
        # if it's time to eat
        if current_time > bird["next_foraging_time"]:
            action_forage(bird["id"], current_time)
        # if the bower needs repair
        elif bird["bower_state"] < 0.0:
            # go repair
            action_repair_bower(bird["id"], current_time)
        # check if it wants to maraud
        elif numpy.random.rand() < bird["probability_maraud"]:
            # go maraud
            action_travel_to_maraud(bird["id"], current_time)
        else:
            # stay at bower
            action_stay_at_bower(bird["id"], current_time)
       

In [7]:
def initialize_bird(bird_id, bird_strategy, bird_xy, bird_preferences, bird_travel_times):
    # initialize dictionary
    bird = {"id": bird_id,
            "current_state": "none",
            "action_starts": 0.0,
            "action_ends": -1.0,
            "probability_maraud": bird_strategy,
            "bower_state": 0.0,
            "successful_mating": 0,
            "next_foraging_time": draw_foraging_time(0.0),
            "travel_preferences": numpy.cumsum(bird_preferences), # note: store cumulative probability for faster choice
            "travel_times": bird_travel_times, 
            "position": bird_xy
            }
    return(bird)

In [8]:
def initialize_female(female_id):
    #initialize dictionary
    female_bird= {"id": female_id,
             "already_visited":[0], #numpy.random.choice(list(range(males)))
             "max_per_day":2,
             "wait_period":12,         
            }
    return(female_bird)

In [9]:
# this is the most important function!
def read_ticket(tic):
    global birds
    if tic["action"] in ("foraging", "staying at bower", "repairing bower", "travel from maraud"):
        # I am back at the bower, choose new action
        choose_action(birds[tic["owner"]], tic["end_time"])
    elif tic["action"] == "travel to maraud":
        # check whether the target is at home
        go_maraud = True
        if birds[tic["target"]]["current_state"] in ("staying at bower", "repairing bower"):
            go_maraud = False
        if birds[tic["target"]]["bower_state"] < 0.0:
            go_maraud = False
        if go_maraud: # maraud
            action_maraud(tic["owner"], tic["target"], tic["end_time"])
        else: # go back
            action_travel_from_maraud(tic["owner"], tic["target"], tic["end_time"])
    elif tic["action"] == "marauding":
        # travel back
        action_travel_from_maraud(tic["owner"], tic["target"], tic["end_time"])
    elif tic["action"] == "mating attempt": #HERE'S AN ADDITION
        if  birds[tic["target"]]["current_state"] == "staying at bower":
            birds[tic["target"]]["successful_mating"]+=1
        else: 
            print("before")
            print(female_birds[tic["owner"]]["already_visited"])
            female_birds[tic["owner"]]["already_visited"].append(tic["target"]) #update the female's already_visited list
            print("after")
            print(female_birds[tic["owner"]]["already_visited"])
            action_mating_attempt(tic["owner"],tic["end_time"])
    else:
        print(tic)
        1 / 0 # something went horribly wrong
    

In [10]:
# Global Variables

# TIMELINE
timeline = SortedDict()
t_max = 12 #12 * 30 # time when simulation ends

# MALES
males = 4 # number of male birds

# FEMALES
female_birds=[]
F_per_M = 3 #The number of sexualy mature females per sexually mature male
females = males*F_per_M # number of female birds
female_visit_param= [0, t_max/2]

# POSITIONS AND TRAVEL TIME
x_dim, y_dim = 400, 500 # dimensions of environment
bird_speed = 12 * 3600 # m/hr (12 m/s)
# now choose lambda_dist, controlling the probability of traveling to a neighbor
# the probability of choosing a neighbor at distance x is proportional to exp(-\lambda x)
# choose lambda such that 99% of the mass is before 800 meters
improb = 0.99
improb_distance = 800
lambda_dist = - math.log(1.0 - improb) / improb_distance

# BIRDS
birds = []
strategies = numpy.random.random(males)

# DEBUG
past_events = []
timeline_lengths = []

######## CODE SIMULATION #########
#numpy.random.seed(0)

# initialize positions, travel times and preferences
positions = generate_positions(males, x_dim, y_dim)
distances, travel_times = compute_distances_travel_times(males, positions, bird_speed)
visit_preferences = compute_visit_preferences(males, distances, lambda_dist)
for i in range(males):
    birds.append(initialize_bird(i, 
                                 strategies[i], 
                                 (positions[0][i], positions[1][i]), 
                                 visit_preferences[i],
                                 travel_times[i]))
    # choose its first action
    choose_action(birds[-1], 0.0)

#initialize females
for i in range(2): #females
    female_birds.append(initialize_female(i+males)) #female IDs start where males end (if there are 10 males, the first female would be 11)
    #choose time for initial mating attempt
    first_time=numpy.random.uniform(female_visit_param[0], female_visit_param[1])
    action_mating_attempt(i, first_time)
                 
# this is the main loop
while len(timeline) > 0:
    print("while")
    current_ticket = timeline.popitem(0)
    read_ticket(current_ticket[1])
    # for debug: store all the past tickets
    past_events.append(current_ticket)
    # and every so often check the length of the timeline
    if numpy.random.rand() < 0.01:
        timeline_lengths.append(len(timeline))


last_loc
0
p:
[0.         0.3630622  0.82596666 1.        ]
last_loc
0
p:
[0.         0.3630622  0.82596666 1.        ]
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
before
[0]
after
[0, 2]
last_loc
2
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while
while

In [11]:
# DEBUG: check that timeline does not bloat!
print(timeline_lengths)

[6]


In [12]:
# DEBUG: MAKE SURE THAT TIMELINE MAKES SENSE
for my_bird in range(males):
    print("#################################")
    print("TIMELINE OF BIRD", my_bird)
    print("#################################")
    bird_actions = {}
    for event in past_events:
        tic = event[1]
        if tic["owner"] == my_bird:
            print(round(tic["start_time"], 3), "-", round(tic["end_time"], 3),
                 "->", tic["action"], "[target:", tic["target"], "]")
    print("#################################")
    print("")
    print("") 
    

    

#################################
TIMELINE OF BIRD 0
#################################
0.0 - 0.205 -> staying at bower [target: -1 ]
0.205 - 0.466 -> staying at bower [target: -1 ]
0.466 - 0.533 -> staying at bower [target: -1 ]
0.533 - 0.805 -> staying at bower [target: -1 ]
0.805 - 0.884 -> foraging [target: -1 ]
0.884 - 1.232 -> staying at bower [target: -1 ]
1.232 - 1.51 -> foraging [target: -1 ]
1.51 - 1.519 -> travel to maraud [target: 1 ]
1.593 - 1.593 -> mating attempt [target: 2 ]
1.519 - 1.619 -> marauding [target: 1 ]
1.619 - 1.628 -> travel from maraud [target: 1 ]
1.628 - 1.724 -> staying at bower [target: -1 ]
1.724 - 1.809 -> foraging [target: -1 ]
1.809 - 1.851 -> staying at bower [target: -1 ]
1.851 - 1.86 -> travel to maraud [target: 1 ]
1.86 - 1.869 -> travel from maraud [target: 1 ]
1.869 - 2.253 -> staying at bower [target: -1 ]
2.253 - 2.264 -> foraging [target: -1 ]
2.264 - 2.48 -> staying at bower [target: -1 ]
2.48 - 2.542 -> staying at bower [target: -1 ]
2.54

9.367 - 9.399 -> foraging [target: -1 ]
9.399 - 9.408 -> travel to maraud [target: 0 ]
9.408 - 9.417 -> travel from maraud [target: 0 ]
9.417 - 9.432 -> staying at bower [target: -1 ]
9.432 - 9.75 -> staying at bower [target: -1 ]
9.75 - 9.798 -> foraging [target: -1 ]
9.798 - 9.856 -> staying at bower [target: -1 ]
9.856 - 9.861 -> travel to maraud [target: 3 ]
9.861 - 9.866 -> travel from maraud [target: 3 ]
9.866 - 9.871 -> travel to maraud [target: 3 ]
9.871 - 9.876 -> travel from maraud [target: 3 ]
9.876 - 9.992 -> staying at bower [target: -1 ]
9.992 - 10.139 -> staying at bower [target: -1 ]
10.139 - 10.413 -> foraging [target: -1 ]
10.413 - 10.551 -> staying at bower [target: -1 ]
10.551 - 10.589 -> staying at bower [target: -1 ]
10.589 - 10.598 -> travel to maraud [target: 2 ]
10.598 - 10.607 -> travel from maraud [target: 2 ]
10.607 - 10.616 -> travel to maraud [target: 0 ]
10.616 - 10.625 -> travel from maraud [target: 0 ]
10.625 - 10.634 -> travel to maraud [target: 2 ]
10

6.385 - 6.661 -> staying at bower [target: -1 ]
6.661 - 6.666 -> travel to maraud [target: 1 ]
6.666 - 6.671 -> travel from maraud [target: 1 ]
6.671 - 6.676 -> travel to maraud [target: 1 ]
6.676 - 6.681 -> travel from maraud [target: 1 ]
6.681 - 6.696 -> staying at bower [target: -1 ]
6.696 - 6.875 -> staying at bower [target: -1 ]
6.875 - 6.941 -> staying at bower [target: -1 ]
6.941 - 7.164 -> staying at bower [target: -1 ]
7.164 - 7.275 -> foraging [target: -1 ]
7.275 - 7.664 -> staying at bower [target: -1 ]
7.664 - 7.672 -> travel to maraud [target: 2 ]
7.672 - 7.68 -> travel from maraud [target: 2 ]
7.68 - 7.705 -> staying at bower [target: -1 ]
7.705 - 7.713 -> travel to maraud [target: 2 ]
7.713 - 7.721 -> travel from maraud [target: 2 ]
7.721 - 7.847 -> staying at bower [target: -1 ]
7.847 - 7.899 -> foraging [target: -1 ]
7.899 - 7.904 -> travel to maraud [target: 1 ]
7.904 - 7.909 -> travel from maraud [target: 1 ]
7.909 - 7.914 -> travel to maraud [target: 1 ]
7.914 - 7.9

In [13]:
for i in range(males):
    print(birds[i]["successful_mating"])

1
1
0
0


In [14]:
print(past_events)

[((0.037025751450630084, 3), {'start_time': 0.0, 'end_time': 0.037025751450630084, 'length_activity': 0.037025751450630084, 'owner': 3, 'action': 'staying at bower', 'target': -1}), ((0.12380075013088748, 2), {'start_time': 0.0, 'end_time': 0.12380075013088748, 'length_activity': 0.12380075013088748, 'owner': 2, 'action': 'staying at bower', 'target': -1}), ((0.17406066710774923, 3), {'start_time': 0.037025751450630084, 'end_time': 0.17406066710774923, 'length_activity': 0.13703491565711914, 'owner': 3, 'action': 'staying at bower', 'target': -1}), ((0.20483881601597173, 0), {'start_time': 0.0, 'end_time': 0.20483881601597173, 'length_activity': 0.20483881601597173, 'owner': 0, 'action': 'staying at bower', 'target': -1}), ((0.2240347881691035, 1), {'start_time': 0.0, 'end_time': 0.2240347881691035, 'length_activity': 0.2240347881691035, 'owner': 1, 'action': 'staying at bower', 'target': -1}), ((0.37666625876433407, 2), {'start_time': 0.12380075013088748, 'end_time': 0.376666258764334

TypeError: 'int' object is not subscriptable

In [16]:
tmp=numpy.random.rand()
p=numpy.cumsum([.1, .2, .3])
thing=numpy.argwhere(p > tmp)
print(thing[0][0])

IndexError: index 0 is out of bounds for axis 0 with size 0

In [17]:
#proving things to ourselves
#random.seed(0)
p = visit_preferences[0] #list of preferences by index (has probability -- not cumulative) #set
av=[0,2] #set
p[av] = 0.0 
p = numpy.cumsum(p)
scale_rand = p[-1:]
tmp = numpy.random.rand() * scale_rand
target = numpy.argwhere(p > tmp)[0][0]
print(target)
# time_to_travel = birds[last_location]["travel_times"][target]
# time_action_ends = current_time + time_to_travel
# generate_ticket(start_time = time_action_ends, #HARD CODE
#                 end_time = time_action_ends,
#                 length_activity = 0,
#                 owner = bird_id,
#                 action = "mating_attempt",
#                 target = target)

1


In [ ]:
print(current_ticket[1])

In [ ]:
 print(females[0])